In [9]:
ATTACK_EPS = 10
ATTACK_STEPSIZE = 0.3
ATTACK_STEPS = 20
NUM_WORKERS = 16
BATCH_SIZE = 4
CUDA_LAUNCH_BLOCKING=1

import torch as ch
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import sys

sys.path.append('./../primate_robustness_mess/robustness_imagenet/')

from robustness.datasets import Faces_data_rich, Faces
from robustness.datasets import DATASETS, ImageNet
from robustness.attacker import AttackerModel
from torch.utils.data import Dataset

from torch.utils.data import DataLoader

from torchvision import models, transforms
from torchvision.io import read_image
from torch import tensor
import re
import os
from robustness.tools.vis_tools import show_image_row

import pickle

ds = Faces_data_rich('./../primate_robustness_mess/PrimateVisionRobustness/Data/Robustness/Stimuli/image_adversarial/pilot_study_v3/')
train_loader, test_loader = ds.make_loaders(workers=NUM_WORKERS, batch_size=BATCH_SIZE)

==> Preparing dataset faces..


In [11]:
device = 'cuda'
# DataLoader
def get_dataloader(csv_file, root_dir, batch_size, transform):
    class CustomDataset(Dataset):
        def __init__(self, csv_file, root_dir, transform=None):
            self.annotations = pd.read_csv(csv_file)
            self.root_dir = root_dir
            self.transform = transform

        def __len__(self):
            return len(self.annotations)

        def __getitem__(self, idx):
            
            img_name = f"{self.root_dir}/{self.annotations.iloc[idx, 8]}"
            try:
                image = Image.open(img_name).convert("L").convert("RGB")
            except:
                print(img_name)
            label1 = torch.tensor([self.annotations.iloc[idx, 3], abs(1 - self.annotations.iloc[idx, 3])])* self.annotations.iloc[idx, 7].squeeze()
            label2 =  self.annotations.iloc[idx, 3]
            if self.transform:
                image = self.transform(image)
            
            return image, (label1, label2)

    dataset = CustomDataset(csv_file=csv_file, root_dir=root_dir, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers = 8)
    return dataloader

# Model Modification
def modify_resnet50(device):
    model = models.resnet50(pretrained=True)
    num_ftrs = model.fc.in_features

    # Modify the fully connected layer to output two sets of classes
    class CustomResNet50(nn.Module):
        def __init__(self, original_model):
            super(CustomResNet50, self).__init__()
            self.features = nn.Sequential(*list(original_model.children())[:-1])
            for param in self.features.parameters():
                param.requires_grad = False

            self.fc1 = nn.Sequential(
                nn.Linear(num_ftrs, 512),
                nn.ReLU(),
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.Linear(512, 2),
                nn.Sigmoid()
                )
            
            self.fc2 = nn.Sequential(
                nn.Linear(num_ftrs, 512),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.Linear(512, 512),
                nn.Linear(512, 256),
                nn.ReLU(),
                nn.Linear(256, 1),
                nn.Sigmoid()
                )
            
            self.linear = nn.Linear(258, 1)

        def forward(self, x):
            x = self.features(x)
            x = x.view(x.size(0), -1)
            out1 = (self.fc1(x)-0.5)*8
            out2 = self.fc2(x)
            out2_expanded = torch.cat((out2, abs(1 - out2)), dim=1)
            out1 = out1 * out2_expanded
            #out2 = nn.Tanh()(self.linear(torch.cat((x, out1), dim=1)))
            return out1, out2   # allows for distance to be between -4 and 4
        
        def freeze(self, freeze = True):
            freeze = not freeze # Invert the freeze parameter to transorm it in require grad
            for param in self.features.parameters():
                param.requires_grad = freeze

                
        def freeze_fc2(self, freeze = True):
            freeze = not freeze # Invert the freeze parameter to transorm it in require grad
            for param in self.fc2.parameters():
                param.requires_grad = freeze

    model = CustomResNet50(model).to(device)
    return model
    


In [16]:
saved_model = modify_resnet50(device)
saved_model = nn.DataParallel(saved_model)
saved_model.load_state_dict(torch.load('trained_two_tailed_resnet50_lr_0.001_epochs_5.pth'))
saved_model.eval()


/tmp/ipykernel_1402359/1199074549.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  saved_model.load_state_dict(torch.load('trained_two_tailed_resnet50_lr_0.001_epochs_5.p

RuntimeError: Error(s) in loading state_dict for DataParallel:
	Missing key(s) in state_dict: "module.fc2.4.weight", "module.fc2.4.bias", "module.fc2.6.weight", "module.fc2.6.bias". 
	Unexpected key(s) in state_dict: "module.fc2.5.weight", "module.fc2.5.bias". 
	size mismatch for module.fc2.3.weight: copying a param with shape torch.Size([256, 512]) from checkpoint, the shape in current model is torch.Size([512, 512]).
	size mismatch for module.fc2.3.bias: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([512]).

In [ ]:
model = AttackerModel(model = saved_model.to('cuda'), dataset = ds).cuda()
